### Pt0.5 - imports and (mounted) paths

In [1]:
import pandas as pd
import numpy as np
from glob import glob
from os import path,makedirs
import re

In [2]:
bids_dir = '/Users/henrymj/Documents/SRO/tmp/OAK/data/uh2/aim1/BIDS_scans'
deriv_dir = path.join(bids_dir, 'derivatives')
firstlevel_path = path.join(deriv_dir, '1stlevel')
fsl_input_path = path.join(deriv_dir, 'fsl/inputs')
fsl_output_path = path.join(deriv_dir, 'fsl/outputs')

In [ ]:
#making some dirs that I didn't get around to

In [3]:
curr_fsl_dirs = glob(path.join(fsl_output_path, 's*', '*'))
curr_fsl_dirs.sort()

In [ ]:
for dpath in curr_fsl_dirs:
    if path.isdir(dpath):
        print(dpath)
        makedirs(path.join(dpath, 'RT-True'), exist_ok=True)
        makedirs(path.join(dpath, 'RT-False'), exist_ok=True)

## Pt1 - making simple events .tsv files for design matrix

In [10]:
eventsfiles = glob(path.join(firstlevel_path, '*/*', 'simplified_events_RT-True_*')) #the RT regressor does not affect the calculation of other regressors
eventsfiles.sort()

In [ ]:
for ef in eventsfiles:
    subjtask = ef.split('1stlevel/')[1].split('/simplified')[0] #get subj/task
    makedirs(path.join(fsl_input_path, subjtask), exist_ok=True) #make fsl/inputs/subj/task dir
    makedirs(path.join(fsl_output_path, subjtask), exist_ok=True) #make fsl/inputs/subj/task dir
    ev_df = pd.read_csv(ef)
    curr_conditions = ev_df.conditions.unique()
    for cond in curr_conditions:
        tmp = ev_df[ev_df.conditions==cond] #get condition subset
        tmp = tmp.filter(['onsets', 'durations', 'amplitudes']) #drop irrelevant columns
        tmp.to_csv(path.join(fsl_input_path, subjtask, cond+'_events.tsv'), sep='\t', header=False, index=False) #save as simple tsv

## Pt2 - making confound .tsv files for design matrix

In [3]:
confound_cols = ['trans_*', 'rot_*', 'framewise_displacement', 'a_comp_cor_*', 'rejectTR_*', 'drift_*', 'constant']

In [26]:
designfiles = glob(path.join(firstlevel_path, '*/DPX', 'design_RT-True*')) #confounds should not be affected by presence/absence of RT regressor
designfiles.sort()

In [31]:
for des in designfiles:
    subjtask = des.split('1stlevel/')[1].split('/design')[0] #get subj/task
    makedirs(path.join(fsl_input_path, subjtask), exist_ok=True) #make fsl/inputs/subj/task dir
    des_df = pd.read_csv(des, index_col=0)
    confound_df = des_df.filter(regex='|'.join(confound_cols))
    confound_df.to_csv(path.join(fsl_input_path, subjtask, 'confounds.tsv'), sep='\t', header=False, index=False) #save as simple tsv    

In [30]:
confound_df

,trans_x,trans_y,trans_z,rot_x,rot_y,rot_z,trans_xtd,trans_ytd,trans_ztd,rot_xtd,...,drift_9,drift_10,drift_11,drift_12,drift_13,drift_14,drift_15,drift_16,drift_17,constant
0.00,0.000039,0.118282,0.024608,-0.001136,2.678020e-23,0.000123,-0.007885,-0.089178,-0.038953,0.000376,...,0.042753,0.042753,0.042752,0.042751,0.042749,0.042748,0.042747,0.042746,0.042744,1.0
0.68,-0.007846,0.029104,-0.014345,-0.000760,-0.000000e+00,0.000000,-0.002707,-0.059061,0.003306,0.000618,...,0.042725,0.042717,0.042709,0.042700,0.042690,0.042679,0.042668,0.042655,0.042642,1.0
1.36,-0.005376,0.000160,0.031220,0.000100,-2.732250e-04,0.000153,-0.002014,-0.003025,0.018778,0.000003,...,0.042668,0.042647,0.042624,0.042598,0.042571,0.042541,0.042509,0.042475,0.042439,1.0
2.04,-0.011875,0.023054,0.023211,-0.000753,-0.000000e+00,0.000201,-0.002402,0.064087,0.022229,0.000087,...,0.042582,0.042541,0.042496,0.042446,0.042392,0.042334,0.042272,0.042205,0.042134,1.0
2.72,-0.010180,0.128335,0.075679,0.000273,1.631110e-23,0.000301,0.000610,0.008472,-0.012135,0.000081,...,0.042468,0.042400,0.042326,0.042244,0.042155,0.042059,0.041956,0.041846,0.041729,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740.52,0.264875,1.970230,-0.259777,0.011574,-9.018320e-03,-0.006945,0.002393,-0.057745,0.016080,0.000027,...,-0.042468,0.042400,-0.042326,0.042244,-0.042155,0.042059,-0.041956,0.041846,-0.041729,1.0
741.20,0.266518,1.943130,-0.245802,0.011643,-9.047750e-03,-0.006848,-0.000404,0.026305,0.020149,-0.000023,...,-0.042582,0.042541,-0.042496,0.042446,-0.042392,0.042334,-0.042272,0.042205,-0.042134,1.0
741.88,0.264067,2.022840,-0.219479,0.011529,-9.042610e-03,-0.006838,-0.000167,0.070970,0.000980,0.000013,...,-0.042668,0.042647,-0.042624,0.042598,-0.042571,0.042541,-0.042509,0.042475,-0.042439,1.0
742.56,0.266184,2.085070,-0.243842,0.011669,-8.973640e-03,-0.007105,0.002858,-0.011975,-0.022345,-0.000005,...,-0.042725,0.042717,-0.042709,0.042700,-0.042690,0.042679,-0.042668,0.042655,-0.042642,1.0


## Pt3 - generate slicetiming files

In [126]:
import json as JSON

In [170]:
task_jsons = glob(path.join(bids_dir, '*_bold.json'))

In [174]:
for curr_json in task_jsons:
    #read in json dict
    jf = open(curr_json,'r')
    task_dict = JSON.loads(jf.read())
    jf.close()
    #get relevant path
    task_str = curr_json.split('task-')[1].replace('_bold.json', '')
    out_slice_file = path.join(fsl_input_path, task_str+'_slice_times.txt')
    #write the slice times into a file
    f=open(out_slice_file, 'w+')
    for slice_time in task_dict['SliceTiming']:
        f.write(str(slice_time)+'\n')
    f.close()

/Users/henrymj/Documents/SRO/tmp/OAK/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/stroop_slice_times.txt
/Users/henrymj/Documents/SRO/tmp/OAK/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/motorSelectiveStop_slice_times.txt
/Users/henrymj/Documents/SRO/tmp/OAK/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/rest_slice_times.txt
/Users/henrymj/Documents/SRO/tmp/OAK/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/twoByTwo_slice_times.txt
/Users/henrymj/Documents/SRO/tmp/OAK/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/discountFix_slice_times.txt
/Users/henrymj/Documents/SRO/tmp/OAK/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/CCTHot_slice_times.txt
/Users/henrymj/Documents/SRO/tmp/OAK/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/ANT_slice_times.txt
/Users/henrymj/Documents/SRO/tmp/OAK/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/DPX_slice_times.txt
/Users/henrymj/Documents/SRO/tmp/OAK/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/surveyMedley_slice_times.txt
/Users/henr

## Pt4 - extract brains

this will generate commands like the one below and save them to an sbatch_file in SRO_fMRI/fmri_analyses/fsl

`/share/software/user/open/fsl/5.0.10/bin/bet /oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/sub-s061/ses-1/anat/sub-s061_ses-1_T1w /oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/s061/brain  -f 0.5 -g 0 -m
`

In [84]:
sbatch_headers='''#!/bin/bash
#SBATCH --job-name=fsl_BET
#SBATCH --output=.out/fsl_BET.out
#SBATCH --error=.err/fsl_BET.err
#SBATCH --time=10:00:00
#SBATCH --mail-type=END
#SBATCH --mail-user=poldracklab@stanford.edu
#SBATCH --cpus-per-task=8
#SBATCH --ntasks-per-node=1
#SBATCH -p russpold
export PYTHONPATH=""
'''

In [85]:
fsl_command = '/share/software/user/open/fsl/5.0.10/bin/bet'
sherlock_bids = '/oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans'
sherlock_fsl = '/oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs'
fsl_flags = '  -f 0.5 -g 0 -m'

In [86]:
anats = glob(path.join(bids_dir, 'sub-*', 'ses-*', 'anat', '*_T1w*'))
anats.sort()

In [94]:
f=open('../fsl/fsl_BET.batch', 'w+')

In [95]:
f.write(sbatch_headers)
f.write('\n')

1

In [ ]:
for anat in anats:
    full_sub = anat.split('BIDS_scans/')[1].split('/ses')[0]
    subid = full_sub.split('-')[1]

    sesanat = anat.split(full_sub)[1]
    sesanat = sesanat.strip('/')
    
    sesnums = re.findall(r'\d', sesanat)
    sesnum = sesnums[0]

    t1_file = anat.split('/')[-1].replace('.nii.gz', '')

    full_command = fsl_command+' '+path.join(sherlock_bids, full_sub, sesanat, t1_file)+' '+path.join(sherlock_fsl, subid, 'brain'+sesnum)+fsl_flags+'\n'
#     print(full_command)
    f.write(full_command)
    f.write('\n')

In [97]:
f.close()

## P5 - building up bash scripts for each task

In [4]:
import nibabel as nib

In [5]:
def get_relative_brain(task, subject, ses_num):
    possible_brains = glob(path.join(fsl_input_path, subject, 'brain*_mask.nii.gz'))
    if path.exists(path.join(fsl_input_path, subject, 'brain%s.nii.gz' % ses_num)): #if there is a brain for the session, use the appropiate brain
        return path.join(fsl_input_path, subject, 'brain%s.nii.gz' % ses_num)
    else: #use the default brain
        return possible_brains[0].replace('_mask', '')

In [6]:
# tasks = ['ANT', 'CCTHot', 'DPX', 'WATT3', 'discountFix', 'motorSelectiveStop', 'stopSignal', 'stroop', 'twoByTwo']
tasks= ['DPX']

doover_subjects = ['s512', 's471', 's640', 's573', 's581', 's624', 's638', 's615', 's611', 's644', 's627', 's649', 's626', 's635', 's650']

In [8]:
for sub in ['s512', 's471', 's499']:
    if sub not in doover_subjects:
        continue
    print('didnt continue')

didnt continue
didnt continue


#build up bash scripts to run FEAT for different tasks

In [9]:
sh_headers='''fsl_dir="/home/groups/russpold/uh2_analysis/Self_Regulation_Ontology_fMRI/fmri_analysis/fsl"
template_dir=$fsl_dir/"templates"
tmp_dir=$fsl_dir/"tmp_batch"

'''

In [10]:
for task in tasks:
    print(task)
    task_niftis = glob(path.join(bids_dir, 'sub-*', '*/func/*task-%s*_bold.nii.gz' % task))
    task_niftis.sort()

    loop_str = ''
    for curr_nifti in task_niftis:
        #break down path to get {RELATIVE_BOLD}, {SUBJECT}, {SES_BRAIN}, and {NTP}
        relative_bold = curr_nifti.replace(bids_dir, '').replace('.nii.gz', '')
        subject = relative_bold.replace('/sub-', '').split('/')[0]
        if subject not in doover_subjects: #skip already passed subjects
            continue
        ses_num = relative_bold.split('/ses-')[1].split('/')[0]
        try:
            BET_brain_path = get_relative_brain(task, subject, ses_num)
            ses_brain = BET_brain_path.split('/')[-1].replace('.nii.gz', '')
            img = nib.load(curr_nifti)
            shape = img.header.get_data_shape()
            ntp = shape[-1]
            totV = np.prod(shape)

            loop_str += ' %s,%s,%s,%s,%s' % (relative_bold, subject, ses_brain, ntp, totV) #THIS IS THE IMPORTANT BIT
        except IndexError:
            print('check out the brain for %s' % subject)

    potential_str = "OLDIFS=$IFS; IFS=',';\nfor i in"+loop_str+'; do set -- $i;\n' +"""\tsed -e "s|{RELATIVE_BOLD}|$1|g" -e "s|{SUBJECT}|$2|g" -e "s|{SES_BRAIN}|$3|g" -e "s|{NTP}|$4|g" -e "s|{TOT_VOX}|$5|g" $template_dir/template_%s_RT-True_fsl.fsf > $tmp_dir/%s_$2_RT-True_fsl.fsf;
    sed -e "s|{RELATIVE_BOLD}|$1|g" -e "s|{SUBJECT}|$2|g" -e "s|{SES_BRAIN}|$3|g" -e "s|{NTP}|$4|g" -e "s|{TOT_VOX}|$5|g" $template_dir/template_%s_RT-False_fsl.fsf > $tmp_dir/%s_$2_RT-False_fsl.fsf;
    sed -e "s|{SUBJECT}|$2|g" -e "s|{TASK}|%s|g" $template_dir/template_1stlevel_FEAT.batch > $tmp_dir/%s_$2_FEAT.batch;
    sbatch $tmp_dir/%s_$2_FEAT.batch;\ndone;\nIFS=$OLDIFS;
    """ % (task, task, task, task, task, task, task)
    f=open('../fsl/run_%s_FEAT_do_over.sh' % task, 'w+')
    f.write(sh_headers)
    f.write('\n')
    f.write(potential_str)
    f.close()

DPX
